### Ejercicio

Proyecto a resolver: dada la siguiente página

https://www.juntadeandalucia.es/agriculturaypesca/ifapa/ria/servlet/FrontController

nos interesa extraer automáticamente los últimos datos de las estaciones metereológicas de todas las provincias.

Por ejemplo, si pinchamos en la página anterior en Granada obtenemos:

https://www.juntadeandalucia.es/agriculturaypesca/ifapa/ria/servlet/FrontController?action=Static&url=listadoEstaciones.jsp&c_provincia=18

Si ahora pinchamos en la primera estación metereológica (Almuñécar) tenemos:

https://www.juntadeandalucia.es/agriculturaypesca/ifapa/ria/servlet/FrontController?action=Static&url=coordenadas.jsp&c_provincia=18&c_estacion=9

Y si finalmente pinchamos en "Últimos datos registrados":

https://www.juntadeandalucia.es/agriculturaypesca/ifapa/ria/servlet/FrontController?action=Static&url=datosDiarios.jsp&c_provincia=18&c_estacion=9

De ahí queremos extraer los datos **provincia**, **estación**, **ETo Día** y **ETo Acumulada**.

Recuerda que queremos extraer esos datos para todas las estaciones metereológicas de las provincias seleccionadas.

**El objetivo es crear código que haga scraping automático de esos datos y genere un dataframe con la siguiente estructura, que será grabado a csv como datos_estaciones.csv:**

In [1]:
url_base = "https://www.juntadeandalucia.es/agriculturaypesca/ifapa/ria/servlet/FrontController?action=Static&url=datosDiarios.jsp&c_provincia={}&c_estacion={}"
print(url_base.format(50,60))

https://www.juntadeandalucia.es/agriculturaypesca/ifapa/ria/servlet/FrontController?action=Static&url=datosDiarios.jsp&c_provincia=50&c_estacion=60


In [2]:
provincia = 14
estacion= 4

In [3]:
url_estacion = url_base.format(provincia, estacion)
print(url_estacion)

https://www.juntadeandalucia.es/agriculturaypesca/ifapa/ria/servlet/FrontController?action=Static&url=datosDiarios.jsp&c_provincia=14&c_estacion=4


In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
r = requests.get(url_estacion)
r.status_code

200

In [6]:
soup = BeautifulSoup(r.text)

In [7]:
tabla = soup.find("table")

In [8]:
tabla

<table summary="Últimos datos registrados" width="100%">
<tr>
<td class="tituloCampo">Provincia</td>
<td class="textoCampo">Córdoba</td>
<td class="tituloCampo">Fecha de Medida</td>
<td class="textoCampo" nowrap="nowrap">21-06-2020</td>
</tr>
<tr>
<td class="tituloCampo">Estación</td>
<td class="textoCampo">Hornachuelos</td>
<td class="tituloCampo">Día Juliano</td>
<td class="textoCampo">173</td>
</tr>
<tr>
<td colspan="4"> </td>
</tr>
<tr>
<td class="tituloCampo">Temp. Máxima (ºC)</td>
<td class="textoCampo">38.1</td>
<td class="tituloCampo">Velocidad Viento Media (m/s)</td>
<td class="textoCampo">0.5</td>
</tr>
<tr>
<td class="tituloCampo">Hora Temp. Máxima</td>
<td class="textoCampo">15:00</td>
<td class="tituloCampo">Dirección Viento Media (º)</td>
<td class="textoCampo">227.8</td>
</tr>
<tr>
<td class="tituloCampo">Temp. Mínima (ºC)</td>
<td class="textoCampo">17.3</td>
<td class="tituloCampo">Radiación (MJ/m<sup>2</sup>)</td>
<td class="textoCampo">30.1</td>
</tr>
<tr>
<td class=

In [9]:
def my_func(tabla, clave):
    return tabla.find("td",
                      text=clave).find_next_sibling("td").text

In [10]:
my_func(tabla, "ETo (mm/dia)")

'5.89'

In [11]:
provincia = my_func(tabla, "Provincia")
estacion  = my_func(tabla, "Estación")
EToDia    = my_func(tabla, "ETo (mm/dia)")
EToAcu    = my_func(tabla, "ETo Acumulada (mm)")

provincia, estacion, EToDia, EToAcu

('Córdoba', 'Hornachuelos', '5.89', '731.53')

In [12]:
claves = ["Provincia", "Estación", "ETo (mm/dia)",
          "ETo Acumulada (mm)"]

aux = [my_func(tabla,texto) for texto in claves]
provincia, estacion, EToDia, EToAcu = aux

provincia, estacion, EToDia, EToAcu

('Córdoba', 'Hornachuelos', '5.89', '731.53')

In [13]:
diccionario = {"Provincia": provincia,
               "Estación": estacion,
               "ETo (mm/dia)":EToDia,
               "ETo Acumulada (mm)":EToAcu}

diccionario

{'Provincia': 'Córdoba',
 'Estación': 'Hornachuelos',
 'ETo (mm/dia)': '5.89',
 'ETo Acumulada (mm)': '731.53'}

In [14]:
url_principal = "https://www.juntadeandalucia.es/agriculturaypesca/ifapa/ria/servlet/FrontController"

r = requests.get(url_principal)
soup = BeautifulSoup(r.text)

In [15]:
len(soup.find_all("table"))

8

In [16]:
links_estaciones = []
tablas = soup.find_all("table")

for tabla in tablas:
    for link in tabla.find_all("a"):
        links_estaciones.append(link.get("href").split("&")[2:])

In [17]:
import pandas as pd

In [18]:
resultados = pd.DataFrame(columns=claves)
resultados

,Provincia,Estación,ETo (mm/dia),ETo Acumulada (mm)


In [19]:
datos_estaciones = []

for link in links_estaciones:
    codigo_provincia = link[0].split("=")[-1]
    codigo_estacion = link[1].split("=")[-1]
    #print(codigo_provincia, codigo_estacion)
    
    url_estacion = url_base.format(codigo_provincia,
                                   codigo_estacion)
    r = requests.get(url_estacion)
    soup = BeautifulSoup(r.text)
    
    tabla = soup.find("table")
    
    try:
        provincia = my_func(tabla, "Provincia")
        estacion  = my_func(tabla, "Estación")
        EToDia    = my_func(tabla, "ETo (mm/dia)")
        EToAcu    = my_func(tabla, "ETo Acumulada (mm)")
        
        dicc = {"Provincia": provincia,
                "Estación": estacion,
                "ETo (mm/dia)": EToDia,
                "ETo Acumulada (mm)": EToAcu}
        
        resultados = resultados.append(dicc, ignore_index=True)
        
        print(provincia, estacion, EToDia, EToAcu)
    except:
        pass
    
    

Almería Adra 4.94 883.05
Almería Almería 5.14 833.06
Almería Cuevas de Almanzora 5.58 859.18
Almería Fiñana 7.27 1013.1
Almería Huércal-Overa 6.53 882.54
Almería La Mojonera 6.08 874.19
Almería Níjar 6.31 896.8
Almería Níjar 6.44 0.0
Almería Tabernas 7.23 918.64
Almería Tíjola 7.03 853.56
Almería Tíjola 2.19 0.0
Almería Virgen de Fátima-Cuevas de Almanzora 6.61 956.28
Cádiz Basurta-Jerez de la Frontera 7.23 922.07
Cádiz Conil de la Frontera 6.18 799.68
Cádiz IFAPA Centro de Chipiona 5.74 827.25
Cádiz Jerez de la Frontera 6.38 807.59
Cádiz Jimena de la Frontera 6.46 805.85
Cádiz Puerto de Santa María 8.25 747.83
Cádiz Puerto de Santa María 2.75 0.0
Cádiz Sanlúcar de Barrameda 1.67 0.0
Cádiz Sanlúcar de Barrameda 7.73 918.16
Cádiz Sanlúcar de Barrameda 0.92 0.0
Cádiz Vejer de la Frontera 7.22 910.69
Cádiz Villamartín 5.05 885.83
Córdoba Adamuz 5.48 706.69
Córdoba Baena 6.38 852.63
Córdoba Bélmez 8.16 899.06
Córdoba Córdoba 6.48 911.91
Córdoba El Carpio 6.86 902.02
Córdoba Hornachuelos 5.

In [20]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [21]:
resultados

,Provincia,Estación,ETo (mm/dia),ETo Acumulada (mm)
0,Almería,Adra,4.94,883.05
1,Almería,Almería,5.14,833.06
2,Almería,Cuevas de Almanzora,5.58,859.18
3,Almería,Fiñana,7.27,1013.1
4,Almería,Huércal-Overa,6.53,882.54
5,Almería,La Mojonera,6.08,874.19
6,Almería,Níjar,6.31,896.8
7,Almería,Níjar,6.44,0.0
8,Almería,Tabernas,7.23,918.64
9,Almería,Tíjola,7.03,853.56
